<a href="https://colab.research.google.com/github/abhisekde96/fii-pro-analysis/blob/main/CP_positions_D30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:


import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pytz
import os
from multiprocessing import Pool
import time

start_time = time.time()

# Function to load or cache the CSV
def load_or_cache_csv(date):
    local_path = f'./cache/fao_participant_oi_{date}.csv'
    if os.path.exists(local_path):
        print(f"Loading from cache: {date}")
        df = pd.read_csv(local_path)
    else:
        try:
            #print(f"Downloading for: {date}")
            df = pd.read_csv(f'http://archives.nseindia.com/content/nsccl/fao_participant_oi_{date}.csv', header=1)
            os.makedirs('./cache', exist_ok=True)  # Create cache directory if it doesn't exist
            df.to_csv(local_path, index=False)
        except Exception as e:
            print(f"Failed to load data for date: {date} - {str(e)}")
            return None
    return df

# Get current time in IST timezone
ist_timezone = pytz.timezone('Asia/Kolkata')
current_time = datetime.now(ist_timezone)

# Check if current time has crossed 9 PM
if current_time.hour >= 21:  # 21 is 9 PM in 24-hour format
    # If it has crossed 9 PM, today's date is considered
    today = current_time.date()
else:
    # If it hasn't crossed 9 PM, consider yesterday's date as today's date
    today = current_time.date() - timedelta(days=1)

# Collect the last 30 weekdays (excluding weekends)
dates_list = []
date_count = 0
while date_count < 100:
    # Check if the current date is a weekday
    if today.weekday() < 5:  # Monday is 0, Sunday is 6
        # Format and append the date to the list
        dates_list.append(today.strftime('%d%m%Y'))
        date_count += 1
    # Move to the previous day
    today -= timedelta(days=1)

# Load CSV files in parallel using multiprocessing
with Pool() as pool:
    results = pool.map(load_or_cache_csv, dates_list)

# Filter out unsuccessful loads and create dataframes dict
dataframes = {date: df for date, df in zip(dates_list, results) if df is not None}

# Reverse the list for proper date order
successful_dates = [date for date, df in zip(dates_list, results) if df is not None]
dates = pd.DataFrame(successful_dates, columns=['Date'])
dates = dates.iloc[::-1]

# Convert the dictionary values to a list of DataFrames
dataframes_list = list(dataframes.values())

# Dynamically create and process variables t0, t1, t2, etc.
for i in range(len(dataframes_list)):
    df = dataframes_list[i]
    df = df.iloc[2:4, [0, 5, 6, 7, 8]]
    df.set_index('Client Type', inplace=True)
    df[f'c{i}'] = df['Option Index Call Long'] - df['Option Index Call Short']  # Compute call difference
    df[f'p{i}'] = df['Option Index Put Long'] - df['Option Index Put Short']  # Compute put difference
    df = df.iloc[:, [4, 5]]  # Keep only the relevant columns (c and p columns)
    globals()[f't{i}'] = df

# Collect all processed DataFrames into a list
df_list = [globals()[f't{i}'] for i in range(len(dataframes_list))]

# Concatenate all DataFrames along columns (axis=1)
t_all = pd.concat(df_list, axis=1)

# Separate call and put columns
c_columns = [f'c{i}' for i in range(len(dataframes_list))]
calls = pd.concat([t_all[col] for col in c_columns], axis=1)

p_columns = [f'p{i}' for i in range(len(dataframes_list))]
puts = pd.concat([t_all[col] for col in p_columns], axis=1)

# Calculate aggregated call and put positions
agg_dict1 = {col: 'sum' for col in c_columns}
tc = calls.agg(agg_dict1)
rename_dict = {f'c{i}': i for i in range(len(dataframes_list))}
tc = tc.rename(rename_dict)
tc = pd.DataFrame(tc)

agg_dict2 = {col: 'sum' for col in p_columns}
tp = puts.agg(agg_dict2)
rename_dict = {f'p{i}': i for i in range(len(dataframes_list))}
tp = tp.rename(rename_dict)
tp = pd.DataFrame(tp)

# Reverse order and add date columns
tc = tc.iloc[::-1]
tp = tp.iloc[::-1]
tc['Date'] = dates['Date']
tp['Date'] = dates['Date']
tc['Date'] = pd.to_datetime(tc['Date'], format='%d%m%Y').dt.strftime('%d/%m/%Y')
tp['Date'] = pd.to_datetime(tp['Date'], format='%d%m%Y').dt.strftime('%d/%m/%Y')

tc.set_index('Date', inplace=True)
tp.set_index('Date', inplace=True)
tc.rename(columns={0: 'call_positions'}, inplace=True)
tp.rename(columns={0: 'put_positions'}, inplace=True)

# Final DataFrame
all_data = pd.merge(tc, tp, left_index=True, right_index=True, how='inner').reset_index()
all_data = all_data.iloc[::-1]
all_data['Date'] = pd.to_datetime(all_data['Date'], format='%d/%m/%Y')

# Calculate Call/Put Changes and Direction
all_data['Day'] = all_data['Date'].dt.day_name()
all_data['direction'] = all_data.apply(lambda row: 'CALLS>>' if row['call_positions'] > row['put_positions'] else 'PUTS>>', axis=1)
all_data['c1'] = all_data['call_positions'].shift(-1)
all_data['p1'] = all_data['put_positions'].shift(-1)
all_data['cd'] = (all_data['call_positions'] - all_data['c1']) / all_data['c1'] * 100
all_data['pd'] = (all_data['put_positions'] - all_data['p1']) / all_data['p1'] * 100
all_data['cd'] = round(all_data['cd'].abs(), 2).astype(str) + '%'
all_data['pd'] = round(all_data['pd'].abs(), 2).astype(str) + '%'
all_data['CALLS'] = all_data['call_positions'] - all_data['c1']
all_data['PUTS'] = all_data['put_positions'] - all_data['p1']
all_data['CALLS'] = all_data['CALLS'].apply(lambda x: 'ADDED' if x > 0 else 'DUMPED') + ' ' + all_data['cd']
all_data['PUTS'] = all_data['PUTS'].apply(lambda x: 'ADDED' if x > 0 else 'DUMPED') + ' ' + all_data['pd']

final_df = all_data[['Date', 'Day', 'call_positions', 'put_positions', 'CALLS', 'PUTS']]

end_time = time.time()

total_time = end_time - start_time
print(f"Total time taken: {total_time:.2f} seconds")

Loading from cache: 08082025
Loading from cache: 07082025
Loading from cache: 06082025
Loading from cache: 05082025
Loading from cache: 04082025
Loading from cache: 01082025
Loading from cache: 31072025
Loading from cache: 30072025
Loading from cache: 29072025
Loading from cache: 28072025
Loading from cache: 25072025
Loading from cache: 24072025
Loading from cache: 23072025
Loading from cache: 22072025
Loading from cache: 21072025
Loading from cache: 18072025
Loading from cache: 17072025
Loading from cache: 16072025
Loading from cache: 15072025
Loading from cache: 14072025
Failed to load data for date: 27082025 - HTTP Error 404: Not Found
Loading from cache: 26082025
Loading from cache: 25082025
Loading from cache: 22082025
Loading from cache: 21082025
Loading from cache: 20082025
Loading from cache: 19082025
Loading from cache: 18082025
Failed to load data for date: 15082025 - HTTP Error 404: Not Found
Loading from cache: 14082025
Loading from cache: 13082025
Loading from cache: 12082

In [19]:
final_df.tail(10)

,Date,Day,call_positions,put_positions,CALLS,PUTS
9,2025-04-28,Monday,149476,-98339,ADDED 45.83%,DUMPED 451.8%
8,2025-04-25,Friday,102501,27953,DUMPED 47.79%,DUMPED 78.91%
7,2025-04-24,Thursday,196330,132540,ADDED 100.68%,ADDED 437.5%
6,2025-04-23,Wednesday,97830,-39271,DUMPED 13.17%,DUMPED 846.52%
5,2025-04-22,Tuesday,112674,-4149,DUMPED 16.29%,ADDED 78.71%
4,2025-04-21,Monday,134596,-19490,DUMPED 51.32%,DUMPED 111.61%
3,2025-04-17,Thursday,276468,167862,DUMPED 20.33%,DUMPED 19.17%
2,2025-04-16,Wednesday,347019,207685,ADDED 81.85%,ADDED 41.62%
1,2025-04-15,Tuesday,190831,146654,DUMPED 47.41%,DUMPED 67.84%
0,2025-04-11,Friday,362871,456034,DUMPED nan%,DUMPED nan%


In [60]:
import yfinance as yf

# Define NIFTY 50 symbol (Yahoo Finance symbol for NIFTY is ^NSEI)
nifty = yf.Ticker("^NSEI")

# Fetch historical data for a date range
nifty_data = nifty.history(start="2025-01-01", end="2025-08-28")

nifty_data = nifty_data.reset_index()
nifty_data['Date'] = pd.to_datetime(nifty_data['Date']).dt.strftime('%Y-%m-%d')

nifty_data = nifty_data[['Date','Close']]
nifty_data['Previous Close'] = nifty_data['Close'].shift(1)
nifty_data['change%'] = ((nifty_data['Close'] - nifty_data['Previous Close']) / nifty_data['Previous Close'] )* 100

In [62]:
final_df['Date'] = final_df['Date'].astype(str)
nifty_data['Date'] = nifty_data['Date'].astype(str)
final_df1 = final_df.merge(nifty_data, left_on='Date', right_on='Date', how='left')

/tmp/ipython-input-2508302603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Date'] = final_df['Date'].astype(str)


In [66]:
final_df1.head()

,Date,Day,call_positions,put_positions,CALLS,PUTS,Close,Previous Close,change%
0,2025-08-26,Tuesday,-99961,482725,DUMPED 215.6%,ADDED 14.57%,24712.050781,24967.750000,-1.024118
1,2025-08-25,Monday,86474,421318,ADDED 822.24%,DUMPED 21.28%,24967.750000,24870.099609,0.392642
2,2025-08-22,Friday,-11973,535226,ADDED 67.89%,ADDED 47.85%,24870.099609,25083.750000,-0.851748
3,2025-08-21,Thursday,-37287,362013,DUMPED 8.28%,ADDED 33.64%,25083.750000,25050.550781,0.132529
4,2025-08-20,Wednesday,-34437,270879,DUMPED 178.12%,DUMPED 17.36%,25050.550781,24980.650391,0.279818
